<a href="https://colab.research.google.com/github/shinoyom89-bit/ml-scikit/blob/main/MAGIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler # used to centre the data and making it into minus 1 or 1 after centring the data and doing standard deviation
from imblearn.over_sampling import RandomOverSampler # this is used to balance the data to reduce the outlyers to get better prediction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cols = ["flength","fwidth","fsize","fconc","fconc1","fasym","fn3long","fm3trans","falpha","fdist","class"] # making the column for the dataframes here
df = pd.read_csv('/content/magic04.data', names=cols) # using the maked col name for the datframe using the name arrg
df.head()


In [ ]:
df['class']=(df['class']=="g").astype(int) # converting the boolena indexing into the 0 and 1 1 is for g the true condition

In [ ]:
for label in cols[:-1]:
    plt.hist(df[df["class"]==1][label],color="blue",label="gamma",alpha=0.7,density=True) # A histogram groups data into bins (like ranges of values).
# It then plots frequency of values in each bin on the y-axis, and the bin ranges on the x-axis.
    plt.hist(df[df["class"]==0][label],color="red",label="hydron",alpha=0.7,density=True) # 0 is the hydron datas
    # density=True is used when you want to compare distributions fairly, not to handle or reduce outliers.
    plt.title(label)#acha to ye bins mai dicivide karde and y mai usme
    # kitni elemnt ya number uske ocunts dega
    plt.xlabel(label)
    plt.ylabel("probablity")
    plt.legend()
    plt.show()

TRAIN ,TESTS ,VALIDATION DATA SETS HERE

In [ ]:
shuffled = df.sample(frac=1) # shuffel data set here changes row up down in random order
train = shuffled.iloc[:int(0.6*len(df))] #60% data for train
valid = shuffled.iloc[int(0.6*len(df)):int(0.8*len(df))]#20% valid
test  = shuffled.iloc[int(0.8*len(df)):]#20% test data

In [ ]:
def scale_dataset(dataframe,oversample=False): #take df and oversample is by default is flse if we want we can do true and can balance the data
    x=dataframe[cols[:-1]].values # .values convert the pandas dataframe  into the np.array
    y=dataframe[cols[-1]].values #Original y is 1D, like a single row of numbers: [0, 1, 0, 1]
#y.reshape(-1,1) → converts it into a column (vertical)
    scaler=StandardScaler()
    x=scaler.fit_transform(x)
    if oversample:
        ros=RandomOverSampler()
        x,y=ros.fit_resample(x,y) # resize the smaller class
    data = np.hstack((x,y.reshape(-1,1))) # Reshape y to be a column vector before stacking
    return data,x,y # the first array in the output is of data and second array is of the x and third is of y

In [ ]:
train,x_train,y_train=scale_dataset(train,oversample=True) # here we store the data and x and y in the train and x_train and ytrain
valid,x_valid,y_valid=scale_dataset(valid,oversample=False) # we set this to flase to take it as unseen data for our model
test,x_test,y_test=scale_dataset(test,oversample=False) # we take the false here to take it as unseen data for the test
print(train)

[[ 4.3045446   8.89784872  4.98849098 ...  2.21652309 -0.56077256
   0.        ]
 [-0.68143941 -0.19742043 -0.44187435 ... -0.81899927  0.08913304
   1.        ]
 [ 1.74050541  0.79993031  1.34198001 ... -0.92577801  1.79594345
   0.        ]
 ...
 [-0.81819504 -0.50507874 -0.60074987 ...  1.33117894  0.61091017
   0.        ]
 [ 5.95347722  2.62805464  2.16940372 ...  0.62069402 -0.07045857
   0.        ]
 [-0.38079966 -0.15614076  0.06631549 ...  2.07114149 -1.58896126
   0.        ]]


In [ ]:
(y_train == 1).sum()# here y is a series so we just pass where y is 1

np.int64(7436)

In [ ]:
(y_train == 0).sum()

np.int64(7436)

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # to find the nearest neighbour
from sklearn.metrics import classification_report # this tell use the reporrt of the prediction target on test data baesd on the train data

In [ ]:
knn_model=KNeighborsClassifier(n_neighbors=1)
knn_model.fit(x_train,y_train) # give the model training data
# To 1st argument = input data
# 2nd argument = us input ka actual output (jo model ko seekhna hai)

KNeighborsClassifier(n_neighbors=1)

In [ ]:
y_prid=knn_model.predict(x_test) # the  y is the target and x we give to model to get train and find the nearest elemnt usin vector distance
# or the prediction of the target on the training data by using the test data
y_prid

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
y_test

array([0, 1, 1, ..., 1, 1, 1])

In [ ]:
print(classification_report(y_test,y_prid)) # cheking the real target from the test and comparing it to the predicted one
# To 1st argument = input data
# 2nd argument = us input ka actual output (jo model ko seekhna hai)

              precision    recall  f1-score   support

           0       0.76      0.66      0.70      1342
           1       0.83      0.89      0.86      2462

    accuracy                           0.81      3804
   macro avg       0.79      0.77      0.78      3804
weighted avg       0.80      0.81      0.80      3804



NAIVE BIAS

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb_model=GaussianNB()
nb_model.fit(x_train,y_train)

GaussianNB()

In [ ]:
y_pred=nb_model.predict(x_test)
print(classification_report(y_test,y_pred)) # train the

              precision    recall  f1-score   support

           0       0.68      0.40      0.50      1342
           1       0.73      0.90      0.81      2462

    accuracy                           0.72      3804
   macro avg       0.70      0.65      0.65      3804
weighted avg       0.71      0.72      0.70      3804



LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lg_model=LogisticRegression()
lg_model.fit(x_train,y_train)

LogisticRegression()

In [ ]:
y_pred=lg_model.predict(x_test)
print(classification_report(y_test,y_pred)) # train the

              precision    recall  f1-score   support

           0       0.69      0.71      0.70      1342
           1       0.84      0.82      0.83      2462

    accuracy                           0.78      3804
   macro avg       0.76      0.77      0.77      3804
weighted avg       0.79      0.78      0.79      3804



SVM SUPPORT VECTOR MACHINE

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm_model=SVC()
svm_model=svm_model.fit(x_train,y_train)
y_pred=svm_model.predict(x_test) # train it on the xtest
print(classification_report(y_test,y_pred)) # giving the actual target in 1st argg and checking the predict y in 2nd arrg

              precision    recall  f1-score   support

           0       0.83      0.79      0.81      1342
           1       0.89      0.91      0.90      2462

    accuracy                           0.87      3804
   macro avg       0.86      0.85      0.85      3804
weighted avg       0.87      0.87      0.87      3804

